# Исследование надежности заемщиков

Заказ на исследование поступил от кредитного отдела банка. Результаты исследования будут учтены при построении модели *кредитного скоринга* - системы, оценивающей способность потенциального заемщика вернуть кредит банку.

**Цель исследования** - проверить влияет ли семейное положение, количество детей, уровень дохода у клиента и цель получения кредита на наличие задолженностей по возврату кредитов. В ходе исследования проверим несколько гипотез:
1. Существует зависимость между количеством детей и возвратом кредита в срок.
2. Существует зависимость между семейным положением и возвратом кредита в срок.
3. Существует зависимость между уровнем дохода и возвратом кредита в срок.
4. Различные цели кредита также влияют на его возврат в срок.

Данные о клиентах банка выгружены в файл `data.csv`. О данных ничего не известно, поэтому перед проверкой гипотез понадобится обзор данных и при необходимости их предобработка.

Исследование будет проведено в три этапа:
1. Обзор данных.
2. Предобработка данных.
3. Проверка гипотез и формулирование выводов.

---

## Обзор данных

In [1]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и сохранение в df
df = pd.read_csv('/datasets/data.csv')

# получение первых 5 строк таблицы df
display(df.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
# просмотр общей информации о данных в таблице df
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов с различными типами данных: *int64*, *float64* и *object*. Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` —  общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

Нарушений стиля в названиях колонок не обнаружено. Но, количество значений в столбцах `days_employed` и `total_income` отличается от общего, что говорит о пропущенных значениях. Поэтому, прежде чем приступать к проверке гипотез, необходимо устранить проблемы в данных.

---

## Предобработка данных

### Проверка пропусков

Посчитаем количество пропущенных значений в данных методами библиотеки `pandas`.

In [3]:
# подсчет пропусков
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В обоих столбцах пропущено одинаковое количество значений - 2174. Оценим какую долю от общего количества данных составляют пропущенные значения.

In [4]:
# подсчет доли пропусков по отношению ко всем данным
df['days_employed'].isna().sum() / df.shape[0]

0.10099883855981417

Пропущенные значения составляют около 10% от общего количества данных. Так как методы формирования данных в этой таблице не известны, можно предположить различные причины появления данных пропусков:
* клиенты могли не предоставить эти данные в процессе анкетирования и/или подачи заявления на кредит;
* данные могли быть пропущены при занесении в базу данных с нецифровых носителей информации;
* данные могли быть частично утрачены в процессе некорректной выгрузки.

Чтобы не удалять подобный существенный объем данных и использовать их в процессе проверки гипотез, заполним пропущенные данные медианными значениями. Это лучшее решение для количественных переменных по сравнению со средними значениями, так как оно менее чувствительно к наличию выбросов (экстремальных значений в данных).

Произведем проверку данных на наличие аномальных значений.

---

### Проверка данных на аномалии и их исправление

Посмотрим на основные описательные статистики данных. Прежде чем заполнять пропущенные значения необходимо проанализировать существующие данные на какие-либо аномальные значения.

In [5]:
# отображение основных описательных статистик данных
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Анализ полученных статистических метрик позволяет сделать выводы о следующих проблемах с данными, с которыми предстоит поработать:
* отрицательное значение в колонке `children`;
* отрицательные значения в колонке о трудовым стаже `days_employed`;
* аномально большие значения в колонке с трудовым стажем `days_employed`.

Подобные ошибки в данных могут быть связаны с человеческим фактором и ошибками в процессе занесения соотвествующих показателей в базы.

Исправим данные с отрицательным количеством детей, заменив некорректные данные на медианные значения. Для этого оценим количество значений с некорректными данными.

In [6]:
# количество клиентов с "отрицательным" количеством детей
df.loc[df['children'] < 0]['children'].count()

47

Общее количество некорректных данных о детях незначительно и может быть заполненно медианным значением, полученным по оставшимся корректным данным.

In [7]:
# определение медианного значения по количеству детей для всех данных за исключением тех, что имеют отрицательные значения
median_children = int(df.loc[df['children'] >= 0]['children'].median())
median_children

0

In [8]:
# замена отрицательных значений в количестве детей на медианное
df.loc[df.loc[df['children'] < 0].index, 'children'] = median_children

In [9]:
# проверка количества клиентов с "отрицательным" количеством детей после исправления
df.loc[df['children'] < 0]['children'].count()

0

Также посмотрим на респределение клиентов с более чем 5 детьми. Тут также могут находиться аномальные данные.

In [10]:
df.loc[df['children'] > 5]['children'].value_counts()

20    76
Name: children, dtype: int64

76 клиентов с 20 детьми. Это явная ошибка в данных. Также заменим их на медианное значение.

In [11]:
# замена значений у клиентов с 20 детьми в данных на медианное значение
df.loc[df.loc[df['children'] == 20].index, 'children'] = median_children

---

Далее исправим аномалии в данные о днях трудового стажа. Для начала также определим общее количество проблемных данных.

In [12]:
# количество клиентов с "отрицательным" количеством дней трудового стажа
df.loc[df['days_employed'] < 0]['days_employed'].count()

15906

Большая часть данных по трудовому стажу имеет отрицательные значения. Поэтому, подход, использованный выше для количества детей здесь неприменим. Можно отметить систематическое появления знака `-` в данных о трудовом стаже и предположить, что он был получен из знака дефиса при занесении данных в базу. Следовательно, можно попробовать данные о трудовом стаже взять по модулю данного числа.

In [13]:
# перезапись значения трудового стажа в отрицательных днях аналогичным по модулю значением
df.loc[df.loc[df['days_employed'] < 0].index, 'days_employed'] = \
abs(df.loc[df.loc[df['days_employed'] < 0].index, 'days_employed'])

Для проверки корректности подобного подхода предлагается провести следующую проверку. Взять все имеющиеся данные по трудовому стажу, измеряемому в днях, за исключением аномально больших, которые могут быть определены следующим пороговым значением:

`60 * 250 = 15000`, то есть 60 лет трудового стажа на 250 рабочих дней в году.

Разделить их на продолжительность трудового стажа, измеренного в годах, величину которого можно определить как текущий возраст клиента из столбца `dob_years` минус возраст 20 лет как усредненный показатель, с которого может начинаться отсчет трудового стажа.

После этого разделить данные трудового стажа в днях на данные по трудовому стажу в годах и посмотреть какая доля значений превышает условно максимальное количество трудовых дней в году, составляющее примерно 250 дней.

In [14]:
# количество данных по количеству дней трудового стажа за исключением аномально больших
df.loc[df['days_employed'] <= 15000]['days_employed'].count()

15893

In [15]:
# количество аномально больших данных по количеству дней трудового стажа
df.loc[df['days_employed'] > 15000]['days_employed'].count()

3458

In [16]:
# доля аномально больших данных
df.loc[df['days_employed'] > 15000]['days_employed'].count() / df.loc[df['days_employed'] <= 15000]['days_employed'].count()

0.2175800666960297

Аномально больших данных почти 22%. Достаточно много. Но, далее, если остальные данные будут похожи на корректные, аномальные данные можно будет попробовать скорректировать.

Для удобства проверки создадим новую таблицу, к которую занесем только корректные по столбцу `days_employed` данные.

In [17]:
# создание новой таблицы по двум условиям: трудовой стаж менее или равен 15000 дней и возраст более 21 года, чтобы
# избежать отрицательных значений
correct_data = df.loc[(df['days_employed'] <= 15000) & (df['dob_years'] > 20)].reset_index(drop=True)

In [18]:
# добавление нового столбца с данными по стажу в годах как разница между возрастом и 20 годами
correct_data['years_employed'] = correct_data['dob_years'] - 20

In [19]:
# добавление нового столбца с расчетом среднего количества рабочих дней за трудовой стаж
correct_data['employed_days_in_year'] = correct_data['days_employed'] / correct_data['years_employed']
correct_data['employed_days_in_year'] = correct_data['employed_days_in_year'].astype('int')

In [20]:
# просмотр полученной таблицы
display(correct_data.tail())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed,employed_days_in_year
15755,1,2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости,17,138
15756,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,23,196
15757,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,18,117
15758,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,18,172
15759,2,1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля,20,99


In [21]:
# подсчет количества аномально больших значений в таблице по количеству рабочих дней в году за трудовой стаж
correct_data.loc[correct_data['employed_days_in_year'] > 250]['employed_days_in_year'].count()

2639

In [22]:
# подсчет доли аномальных дней по отношению к общему количеству данных
correct_data.loc[correct_data['employed_days_in_year'] > 250]['employed_days_in_year'].count() / correct_data.shape[0]

0.16744923857868022

Таким образом, аномально превышает среднее количество в 250 рабочих дней в году приблизительно 17% данных, что можно считать удовлетворительным для подтверждения правильности выбранного решения о взятии модуля для значений количества рабочих дней в году.

Теперь можно для полученной таблицы с относительно корректными данными найти медианное значение количества рабочих дней в году и уже на его основании пересчитать для аномальных значений трудового стажа в основных данных количество рабочих дней в общем трудовом стаже по формуле:

`СТАЖ [дней] = (ВОЗРАСТ [лет] - 20) * РАБОЧИХ ДНЕЙ В ГОДУ [среднее количество дней]`

Здесь лучше отработает среднее значение, так как от аномальных значений мы избавились при создании таблицы корректных данных (`correct_data`).

In [23]:
# среднее значение рабочих дней в году из таблицы корректных данных
mean_days_in_year = int(correct_data['employed_days_in_year'].mean())
mean_days_in_year

139

Среднее количество рабочих дней, рассчитанное по таблицы корректных данных, получили равным 139. Теперь, используем найденное значение, чтобы скорректировать по приведенной выше формуле аномальные значения количества рабочих дней в году в основной таблице.

In [24]:
# перезапись аномальных значений трудового стажа расчетные значения по среднему количеству рабочих дней в году
df.loc[df.loc[(df['days_employed'] > 15000) & (df['dob_years'] > 20)].index, 'days_employed'] = \
(df.loc[df.loc[(df['days_employed'] > 15000) & (df['dob_years'] > 20)].index, 'dob_years'] - 20) * mean_days_in_year

Проверка данных после произведенных корректировок.

In [25]:
# отображение основных описательных статистик данных
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.470476,3216.203475,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.750534,10947.133301,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,926.624121,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2188.846985,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,4726.000000,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,400992.375704,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Из описательных статистик видно, что данные по количеству детей теперь корректны, а с показателями стажа все еще есть проблемы. Проверим количество аномальных значений.

In [26]:
# количество аномально больших данных по количеству дней трудового стажа
df.loc[df['days_employed'] > 15000]['days_employed'].count()

17

Количество аномальных значений сократилось с 3458 до 17. Посмотрим на эти данные, чтобы принять решение по их дальшейшей обработке.

In [27]:
# визуальная оценка аномальных значений
display(df.loc[df['days_employed'] > 15000])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля
4922,0,336516.005867,0,высшее,0,вдовец / вдова,2,F,пенсионер,1,183556.355624,свой автомобиль
7034,0,366067.781030,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,263121.074528,образование
8061,0,366457.872613,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,61804.271999,высшее образование
10188,0,371665.278622,0,среднее,1,женат / замужем,0,M,пенсионер,0,102621.701671,операции с недвижимостью
12062,0,332185.354511,0,среднее,1,женат / замужем,0,F,пенсионер,0,206718.981389,покупка жилья
12729,0,355109.725856,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,54815.744162,образование


Данных немного и они очень плохого качества. Все клиенты пенсионеры нулевого возраста и с нулевым количеством детей. Так как провести какие-либо уточнения по этим данным нет возможности, то единственным решением остается их удаление.

In [28]:
# удаление строк с оставшимися аномально большими данными по количеству дней трудового стажа и обновление индексов
df = df.drop(df.index[(list(df.loc[df['days_employed'] > 15000].index))]).reset_index(drop=True)

Повторная проверка описательных статистик после удаления аномальных данных.

In [29]:
# отображение основных описательных статистик данных
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21508.000000,19334.000000,21508.000000,21508.000000,21508.000000,21508.000000,1.933400e+04
mean,0.470848,2900.259423,43.327599,0.817277,0.972057,0.080900,1.674477e+05
std,0.750714,2412.394379,12.520481,0.548223,1.420024,0.272688,1.029837e+05
min,0.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,926.249613,33.000000,1.000000,0.000000,0.000000,1.030962e+05
50%,0.000000,2184.647508,42.500000,1.000000,0.000000,0.000000,1.450740e+05
75%,1.000000,4726.000000,53.000000,1.000000,1.000000,0.000000,2.034244e+05
max,5.000000,14920.049805,75.000000,4.000000,4.000000,1.000000,2.265604e+06


Теперь также корректны и данные об общем трудовом стаже в днях и поэтому можно перейти к заполнению пропущенных в данных значений.

---

### Заполнение пропусков

In [30]:
# вычислим медианное значение стажа в количестве рабочих дней
median_days_employed = df['days_employed'].median()
median_days_employed

2184.6475075478165

In [31]:
# вычислим медианное значение заработка
median_total_income = df['total_income'].median()
median_total_income

145073.98616379802

Заполним полученными значениями соотвествующие пропущенные значения.

In [32]:
# заполнение пропусков по трудовому стажу
df['days_employed'] = df['days_employed'].fillna(median_days_employed)

In [33]:
# заполнение пропусков по ежемесячному доходу
df['total_income'] = df['total_income'].fillna(median_total_income)

Убедимся, что были обработаны все пропуски.

In [34]:
# подсчет пропусков
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Пропусков в данных больше нет. Переходим к следующих этапам предобработки данных.

---

### Изменение типов данных

В столбцах с данными по общему трудовому стажу `days_employed` данные представлены в виде вещественных чисел с большим количеством знаком после запятой, что может затруднить последующий анализ. Приведем эти данные к целочисленному типу.

In [35]:
# перевод значений столбца total_income из вещественного типа в целочисленный
df['total_income'] = df['total_income'].astype('int')

Проверим полученный результат.

In [36]:
# вывод первых строк таблицы
display(df.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,4587.000000,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Тип данных успешно изменен. Следующий этап предобрботки - проверка данных на наличие дубликатов.

---

### Удаление дубликатов

Проверим наличие дубликатов в данных с помощью последовательного применения методов *duplicated()* и *sum()*.

In [37]:
# проверка данных на наличие дубликатов
df.duplicated().sum()

54

Видно, что дубликаты присутствуют, но пока непонятно что они из себя представляют. Отобразим таблицу с дубликатами для анализа.

In [38]:
# отображение таблицы с дубликатами
display(df.loc[df.duplicated() != False])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2845,0,2184.647508,41,среднее,1,женат / замужем,0,F,сотрудник,0,145073,покупка жилья для семьи
4178,1,2184.647508,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145073,свадьба
4847,0,2184.647508,60,среднее,1,гражданский брак,1,F,пенсионер,0,145073,свадьба
5552,0,2184.647508,58,среднее,1,гражданский брак,1,F,пенсионер,0,145073,сыграть свадьбу
7802,0,2184.647508,57,среднее,1,гражданский брак,1,F,пенсионер,0,145073,на проведение свадьбы
8576,0,2184.647508,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145073,дополнительное образование
9231,2,2184.647508,34,среднее,1,женат / замужем,0,F,сотрудник,0,145073,покупка жилья для сдачи
9521,0,2184.647508,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145073,операции со своей недвижимостью
9620,0,2184.647508,56,среднее,1,женат / замужем,0,F,пенсионер,0,145073,операции со своей недвижимостью
10454,0,2184.647508,62,среднее,1,женат / замужем,0,F,пенсионер,0,145073,покупка коммерческой недвижимости


Из таблицы видно, что дубликаты получены в результате одинакового значения данных по трудовому стажу `days_employed` и ежемесячному доходу `total_income`. При этом остальные данные различаются.

Это явная ошибка, возникшая при занесении данных в базу. Так как возможности получить корректные данные нет, а общее число данных с дубликатами этого типа невелико относительно общего размера данных, то эти строки можно просто удалить, применив цепочку из двух методов.

In [39]:
# удаление строк с дубликатами и обновление индексов таблицы
df = df.drop_duplicates().reset_index(drop=True)

Перепроверим наличие дубликатов после их удаления.

In [40]:
# повторная проверка данных на наличие дубликатов
df.duplicated().sum()

0

При визуальном просмотре результатов вывода таблицы в примерах выше видно, что присутствуют также неявные дубликаты, как минимум в столбце `education`. Посмотрим на список уникальных значений в данном столбце.

In [41]:
# просмотр уникальных значений в столбце education
df['education'].value_counts()

среднее                13693
высшее                  4706
СРЕДНЕЕ                  772
Среднее                  710
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Можно отметить наличие неявных дубликатов в данных по уровню образования клиента в виде записей признака в различном регистре. От дубликатов подобного рода можно избавиться приведением данного признака к единому регистру. В данном случае к нижнему.

In [42]:
# перевод данных об образовании к нижнему регистру
df['education'] = df['education'].str.lower()

Проверка полученного результата.

In [43]:
# повторный просмотр уникальных значений в столбце education
df['education'].value_counts()

среднее                15175
высшее                  5247
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь в столбце `education` все данные в едином регистре и дубликатов нет. Проверим также другие категориальные данные, в которых могут присутствовать дубликаты подобного рода: `family_status`, `income_type`, `purpose`.

In [44]:
# просмотр уникальных значений в столбце family_status
df['family_status'].value_counts()

женат / замужем          12336
гражданский брак          4163
Не женат / не замужем     2806
в разводе                 1194
вдовец / вдова             955
Name: family_status, dtype: int64

In [45]:
# просмотр уникальных значений в столбце income_type
df['income_type'].value_counts()

сотрудник          11091
компаньон           5080
пенсионер           3820
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [46]:
# просмотр уникальных значений в столбце purpose
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  674
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     645
покупка жилья                             644
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              632
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
строительство недвижимости                619
покупка своего жилья                      619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

В этих столбцах проблем с дубликатами, связанных с записью данных строкового типа в различном регистре, не обнаружено. Только в стилистических целях приведем к нижнему регистру значения в столбце `family_status`.

In [47]:
# перевод данных о семейном положении к нижнему регистру
df['family_status'] = df['family_status'].str.lower()

В данных по цели кредита `purpose` обнаружено большое количество нечетких дублей, которые могут затруднить последующую проверку гипотезы на основе данного признака. Попробуем избавиться от них далее объединив нечеткие дубли в группы по ключевым словам.

В процессе предобработки данных были найдены дубликаты различного рода. С помощью метода *duplicated()* были найдены и методом *drop_duplicates()* удалены явные дубликаты в количественных признаках `days_employed` и `total_income`.

Также были проанализированы категориальные признаки `education`, `family_status`, `income_type`, `purpose` на наличие дубликатов. С помощью метода *value_counts()* были отображены сводные таблицы с подсчетом всех уникальных значений в указанных столбцах. Были обнаружены дубликаты в данных по образованию `education`, которые заключались в записи отдинаковых признаков в различном регистре. От дублактов подобного рода данные были очищены приведением данных в столбце по образованию к единому нижнему регистру методом *str.lower()*.

Отмечена дальнейшая необходимость обработки нечетких дублей в данных по целям кредитов `purpose`.

---

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Уровень образования клиента и семейное положение многократно повторяются в данных, а строки с ними занимают много места. Создадим словари для этих признаков и оптимизируем данные, удалив из основной таблицы многократно повторяющиеся значения.

In [48]:
# создание словаря для уровня образования

education_dict = df[['education', 'education_id']].drop_duplicates().reset_index(drop=True)
display(education_dict)

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [49]:
# создание словаря для семейного положения

family_status_dict = df[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
display(family_status_dict)

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


После создания словарей соотвествия образования и семейного положения предопредленным в таблице категориям, столбцы `education` и `family_status` можно удалить из таблицы.

In [50]:
# удаление столбцов education и family_status из датафрейма
df = df.drop(['education', 'family_status'], axis=1)

In [51]:
# проверка полученного результата
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024.803754,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623.422610,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124.747207,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,4587.000000,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Общий вид таблицы портят данные об общем трудовом стаже `days_employed`, которые представлены в виде вещественных чисел с большим количеством знаком после запятой. Приведем эти данные к целочисленному типу.

In [52]:
# перевод значений столбца total_income из вещественного типа в целочисленный
df['days_employed'] = df['days_employed'].astype('int')

Проверим полученный результат.

In [53]:
# вывод первых строк таблицы
display(df.head())

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,4587,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


В таком виде данные выглядят уже пригодными для последующего анализа и проверки выдвинутых гипотез. Но, перед этим необходимо, как было намечено ранее, поработать с нечеткими дублями в данных по целям кредитов `purpose`, а также добавить в них категоризацию по уровню дохода из столбца `total_income`.

---

### Категоризация дохода

Добавим категориальный признак `total_income_category` для уровня дохода, соотнеся количественные данные по доходу к категориальным меткам на основании следующие диапазонов:
* `E` — для клиентов с ежемесячным доходом в диапазоне 0-30000;
* `D` — для клиентов с ежемесячным доходом в диапазоне 30001-50000;
* `C` — для клиентов с ежемесячным доходом в диапазоне 50001-200000;
* `B` — для клиентов с ежемесячным доходом в диапазоне 200001-1000000;
* `A` — для клиентов с ежемесячным доходом  свыше 1000001.

In [54]:
# функция для категоризации ежемесячного дохода клиента

def get_income_group(row):
    """
    Возвращает метку для ежемесячного дохода клиентов банка, используя правила:
    - метка 'E' для клиентов ежемесячным доходом в диапазоне 0-30000
    - метка 'D' для клиентов ежемесячным доходом в диапазоне 30001-50000
    - метка 'C' для клиентов ежемесячным доходом в диапазоне 50001-200000
    - метка 'B' для клиентов ежемесячным доходом в диапазоне 200001-1000000
    - метка 'A' для клиентов с ежемесячным доходом свыше 1000001
    """
    
    total_income = row['total_income']

    if 0 <= total_income <= 30000:
        return 'E'
    elif 30001 <= total_income <= 50000:
        return 'D'
    elif 50001 <= total_income <= 200000:
        return 'C'
    elif 200001 <= total_income <= 1000000:
        return 'B'
    else:
        return 'A'

In [55]:
# добавление категориальных данных по ежемесячному доходу клиента
df['total_income_category'] = df.apply(get_income_group, axis=1)

Выполненная категоризация дохода по клиентам позволит в дальнейшем произвести проверку гипотезы о зависимости между уровнем дохода и возвратами кредитов в срок.

---

### Категоризация целей кредита

Проанализируем список уникальных значений столбца `purpose`.

In [56]:
# просмотр уникальных значений в столбце по цели получения кредита purpose
df['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  674
покупка коммерческой недвижимости         661
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
жилье                                     645
покупка жилья                             644
покупка жилья для семьи                   638
строительство собственной недвижимости    634
недвижимость                              632
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
строительство недвижимости                619
покупка своего жилья                      619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [57]:
# оценка количества уникальных значений в столбце по цели получения кредита purpose
len(df['purpose'].value_counts())

38

Всего в данных 38 уникальных групп, что затруднит их анализ и выявление закономерностей. При этом, данные содержат неявные дубликаты. Путем анализа данных можно выделить четыре основные категории, которые являются целью получения кредита в банке:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение свадьбы`
* `получение образования`

Будем использовать их в качестве меток категорий, которые добавим к существующей таблице в виде нового столбца `purpose_category`.

Для этого создадим вспомогательную функцию.

In [58]:
# функция для обобщения категоризации целей получения клиентами кредитов

def get_purpose_group(row):
    purpose = row['purpose']

    if 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'авто' in purpose:
        return 'операции с автомобилем'
    elif 'образован' in purpose:
        return 'получение образования'

In [59]:
# обобщение категориальных данных по целям получения клиентами кредитов
df['purpose_category'] = df.apply(get_purpose_group, axis=1)

Проверим насколько корректно отработала функция и не остались ли пропущенными какие-либо цели. Для этого просмотрим список уникальных значений по столбцу `purpose_category`. 

In [60]:
# просмотр уникальных значений в столбце по цели получения кредита purpose
df['purpose_category'].value_counts()

операции с недвижимостью    10806
операции с автомобилем       4305
получение образования        4008
проведение свадьбы           2335
Name: purpose_category, dtype: int64

Получены четыре уникальные категории, что соответствует количеству выявленных общих категорий. Значит, функция отработала корректно. Нечетко категоризированных целей получения кредитов в данных больше нет.

Посмотрим на первые значения итоговой таблицы.

In [61]:
# просмотр итоговой таблицы
df.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,4587,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы


На этом этап предобработки данных завершен. В данных заполнены пропущенные значения, устранены аномальные значения, удалены дубликаты, проведена категоризация числового признака в виде ежемесячного дохода. Можно переходить к проверке гипотез.

---

### Проверка выдвинутых гипотез

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

Проанализируем есть ли зависимость между количеством детей у клиента банка и отсутвствием задержек с возвратом кредита. Для этого сгруппируем данные по количеству детей из столбца `children` таблицы и произведем подсчет количества задолженностей по возврату кредита в срок и их отсутствию.

Для удобства интерпретации полученных результатов сведем их в итоговую таблицу, в которую также добавим расчет соотношения количества клиентов с наличием задолженностей к общему количеству клиентов для каждой группы.

In [62]:
# создадим сводную таблицу группировкой данных методом pivot_table() в так называемую "широкую" таблицу
children_table = df.pivot_table(index=['children'], columns='debt', values='purpose', aggfunc='count')

# подсчитаем долю клиентов с задолженностью для каждой группы для удобства анализа полученных результатов
children_table['rate'] = children_table[1] / (children_table[0] + children_table[1])
display(children_table)

debt,0,1,rate
children,,,
0,13142.0,1071.0,0.075354
1,4365.0,444.0,0.092327
2,1858.0,194.0,0.094542
3,303.0,27.0,0.081818
4,37.0,4.0,0.097561
5,9.0,NaN,NaN


В итоговую таблицу попали пропущенные значения, так как для клиентов с 5 детьми в базе не оказалось записей по невозврату кредитов. Также тип данных изменился на вещественные числа. Исправим возникшие проблемы.

In [63]:
# заполнение пропущенных значений
children_table = children_table.fillna(0)

# замена вещественного типа данных в столбцах 0 и 1 на целочисленный.
children_table[0] = children_table[0].astype('int')
children_table[1] = children_table[1].astype('int')

# переименование столбцов по признаку наличия долга: no - долгов не было, yes - долги есть или были
children_table = children_table.rename(columns={0: 'no', 1: 'yes'})

# просмотр таблицы
display(children_table)

debt,no,yes,rate
children,,,
0,13142,1071,0.075354
1,4365,444,0.092327
2,1858,194,0.094542
3,303,27,0.081818
4,37,4,0.097561
5,9,0,0.000000


Выводы по результатам:
* Минимальная доля клиентов с задолженностями наблюдается у группы без детей (7.5%).
* Среди клиентов с 1 или 2 детьми доли задолжености по кредитам примерно равны и составляют 9.2% и 9.5% соотвественно.
* Показатель по задолженности для клиентов с 3 детьми составил 8.2%, расположившись, таким образом, между бездетными клиентами и клиентами с 1 ребенком или 2 детьми. Но, объем выборки по данной группе составляет около 1.5% от всех данных и может быть нерепрезентативен. 
* Для клиентов с 4 и 5 детьми данных еще меньше, поэтому делать какие-то выводы по данным группам некорректно.
* В целом, разница между отдельными группами клиентов по признаку количества детей составляет составляет около 2%. Статистическую значимость подобного результата необходимо проверять дополнительно.

---

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Проанализируем наличие зависимости между семейным положением клиента банка и отсутвствием задержек с возвратом кредита. Для этого сгруппируем данные по семейному положению из столбца `family_status_id` и произведем подсчет количества задолженностей по возврату кредита в срок и их отсутствию.

Для нагладности результаты сведем в итоговую таблицу, в которой также подсчитаем соотношение количества клиентов с наличием задолженностей к общему количеству клиентов в каждой группе.

In [64]:
# создадим сводную таблицу группировкой данных методом pivot_table() в так называемую "широкую" таблицу
family_status_table = df.pivot_table(index=['family_status_id'], columns='debt', values='purpose', aggfunc='count')

# заменим метки категорий на соответствующие описания
family_status = family_status_dict[['family_status_id', 'family_status']].\
set_index('family_status_id').to_dict()['family_status']
family_status_table = family_status_table.rename(index=family_status)

# подсчитаем долю клиентов с задолженностью для каждой группы для удобства анализа полученных результатов
family_status_table['rate'] = family_status_table[1] / (family_status_table[0] + family_status_table[1])

# переименование столбцов по признаку наличия долга: no - долгов не было, yes - долги есть или были
family_status_table = family_status_table.rename(columns={0: 'no', 1: 'yes'})

# просмотр таблицы
display(family_status_table)

debt,no,yes,rate
family_status_id,,,
женат / замужем,11405,931,0.075470
гражданский брак,3775,388,0.093202
вдовец / вдова,893,62,0.064921
в разводе,1109,85,0.071189
не женат / не замужем,2532,274,0.097648


Выводы по результатам:
* Минимальное значение по невозврату кредитов в срок наблюдается в группе `вдовец / вдова`. Но, данный результат может быть недостоверным, так как выборка по данном группе значительно меньше по сравнению с другими.
* Самые высокие значения по невозврату кредитов наблюдаеются в группах `гражданский брак` и `не женат / не замужем` - 9.3% и 9.8% соотвественно.
* В группах `женат / замужем` и `в разводе` процент невозвращенных вовремя кредитов также очень близок и составляет 7.5% и 7.1% соотвественно.
* Судя по полученным данным, не состоящие в законных отношениях клиенты, несколько менее дисциплинированно относятся к своим обязательствам по кредитам.
* В целом, разница между отдельными группами по семейному положению составляет составляет около 3%. Статистическую значимость подобного результата необходимо проверять дополнительно.

---

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на данный вопрос проанализируем наличие зависимости между уровнем дохода клиента и отсутвствием задержек с возвратом кредита. Для этого произведем подсчет количества задолженностей по возврату кредита в срок и их отсутствию для различных по уровню ежемесячного дохода групп клиентов, которые представлены столбцом `purpose_category` анализируемой таблицы.

Для нагладности результаты сведем в итоговую таблицу, в которой также подсчитаем соотношение количества клиентов с наличием задолженностей к общему количеству клиентов в каждой группе.

In [65]:
# создадим сводную таблицу группировкой данных методом pivot_table() в так называемую "широкую" таблицу
income_table = df.pivot_table(index=['total_income_category'], columns='debt', values='purpose', aggfunc='count')

# заменим метки категорий на соответствующие описания
purpose_dict = {'E': 'доход в диапазоне 0-30000',
                'D': 'доход в диапазоне 30001-50000',
                'C': 'доход в диапазоне 50001-200000',
                'B': 'доход в диапазоне 200001-1000000',
                'A': 'доход свыше 1000001'}
income_table = income_table.rename(index=purpose_dict)

# подсчитаем долю клиентов с задолженностью для каждой группы для удобства анализа полученных результатов
income_table['rate'] = income_table[1] / (income_table[0] + income_table[1])

# переименование столбцов по признаку наличия долга: no - долгов не было, yes - долги есть или были
income_table = income_table.rename(columns={0: 'no', 1: 'yes'})

# просмотр таблицы
display(income_table)

debt,no,yes,rate
total_income_category,,,
доход свыше 1000001,23,2,0.080000
доход в диапазоне 200001-1000000,4680,356,0.070691
доход в диапазоне 50001-200000,14663,1359,0.084821
доход в диапазоне 30001-50000,328,21,0.060172
доход в диапазоне 0-30000,20,2,0.090909


Выводы по результатам:
* Самая низкая задолженность по возврату кредитов около 6% наблюдается в группе клиентов с ежемесячным доходом в диапазоне 30001-50000.
* Самая высокая - 9% в группе клиентов с ежемесячным доходом в диапазоне 0-30000. Но, в данной группе всего 22 наблюдения, поэтому этот показатель нельзя считать репрезентативным для данной группы.
* В целом, какой-либо зависимости между уровнем дохода и возвратом кредита в срок по полученным данным не обнаружено. Разница между отдельными группами составляет около 3% и ее распределение не позволяет выявить значимые закономерности.

---

#### Как разные цели кредита влияют на его возврат в срок?

В завершении исследования проверим как различные цели, под которые берется кредит, влияют на наличие или отсутствие задолженностей по возврату кредита. Для этого произведем подсчет количества задолженностей по возврату кредита в срок и их отсутствию в зависимости от целей кредита для групп из колонки `purpose_category` таблицы.

Для нагладности результаты сведем в таблицу, в которой также подсчитаем соотношение количества клиентов с наличием задолженностей к общему количеству клиентов в каждой группе.

In [66]:
# создадим сводную таблицу группировкой данных методом pivot_table() в так называемую "широкую" таблицу

purpose_table = df.pivot_table(index=['purpose_category'], columns='debt', values='purpose', aggfunc='count')

# подсчитаем долю клиентов с задолженностью для каждой группы для удобства анализа полученных результатов
purpose_table['rate'] = purpose_table[1] / (purpose_table[0] + purpose_table[1])

# переименование столбцов по признаку наличия долга: no - долгов не было, yes - долги есть или были
purpose_table = purpose_table.rename(columns={0: 'no', 1: 'yes'})

# просмотр таблицы
display(purpose_table)

debt,no,yes,rate
purpose_category,,,
операции с автомобилем,3903,402,0.093380
операции с недвижимостью,10024,782,0.072367
получение образования,3638,370,0.092315
проведение свадьбы,2149,186,0.079657


Выводы по результатам:
* Самая низкая задолженность по возврату кредитов наблюдается, когда они берутся под операции с недвижимостью (7.2%).
* Немного выше, примерно 8% показатель задолженности по кредитам на проведение свадьбы.
* Количество задолженностей по кредитам на получение образования и операции с автомобилями примерно равно и немногим превышает величину в 9%.
* В целом, значимых различий в показателе возврата кредита в срок в зависимости от его целей не обнаружено. Эта разница не превышает 1-2%.

---

## Общие выводы по исследованию

3.1. В ходе исследования была осуществлена попытка анализа и выявления зависимости между возвратом кредитов клиентами банка в срок от различных факторов таких как количество детей, семейное положение, уровень дохода и цели получения кредита.


В полученных для проведения исследования данных был обнаружен ряд проблем:
* Пропущенные значения в столбцах общего трудового стажа `days_imployed` и ежемесячного дохода `total_income` у 10% записей.
* Отрицательные и аномальные значения в данных о количестве детей в семье `children` и общем трудовом стаже `days_imployed`.
* Обнаружены 76 клиентов, у которых количество детей равнялось 20.


Данные явно содержали ошибки различного рода и нуждались в предобработке. В ходе предобработки данных были предприняты следующие меры:
* Отрицательные и аномальные значения в данных о количестве детей в семье `children` были заменены медианными.
* Было выдвинуто предположение, что отрицательные значения трудового стажа эквивалентны модулю данных значений. Предположение было подтверждено расчетами, в ходе которых было утановлено, что у 22% клиентов данные по трудовому стажу превышают разумные пределы, определенные 15000 днями трудового стажа. По результатам трудового стажа остальных клиентов было определено медианное количество рабочих дней в году и по нему пересчитаны аномальные значения. Количество аномальных значений сократилось с 3458 клиентов до 17. Оставшиеся 17 записей имели неудовлетворительное качество и были удалены.
* После корректировки аномальных значений пропущенные данные по трудовому стажу `days_imployed` и ежемесячному доходу `total_income` были также заполнены медианными значениями.


3.2. Вещественный тип данных в столбцах общего трудового стажа `days_imployed` и ежемесячного дохода `total_income` с большим количеством знаков после запятой был преобразован в целочисленный. В данных обнаружены дубликаты в количестве 54 записей. Они были получены в результате одинакового значения данных по трудовому стажу `days_employed` и ежемесячному доходу `total_income`. Так как идентификаторов клиентов в полученной таблице нет, то не было возможности проверить дублируются ли данные какого-то одного клиента или информация по разным клиентам совпала каким-либо образом. По этой причине дублированные данные были удалены.


3.3. По уникальным значениям столбцов `education` и `family_status` были созданы словари, а с основным датафреймом была проведена декомпозиция путем удаления данных столбцов с сохранением лишь идентификаторов соотвествующих данных.


3.4. Для последующего анализа была проведена категоризация данных по ежемесячному доходу `total_income`. Были выделены пять групп по величине дохода и соотвествующие им метки добавлены в основной датафрейм.


3.5. Количество категорий целей получения кредита из столбца `purpose` путем удаления неявных дублей удалось сократить с 38 до 4:
* `операции с автомобилем`
* `операции с недвижимостью`
* `проведение свадьбы`
* `получение образования`

Это позволило корретно проивести анализ влияния различных целей кредита на срок его возврата.


3.6. Была проведена проверка гипотезы о влиянии количества детей в семье, семейного положения, величины месячного дохода, а также целей получения кредита на наличие задолженностей по его возврату.

Полученные в процессе иследования данные сведены в наглядные таблицы с расчетом долей клиентов с задолженностями для каждой категории.

По всем исследованным категориям доля клиентов с задолженнностями не превысила 10% порог, а различия между выделенными группами не превышали 3%.

На данном этапе исследования явных закономерностей, влияющих на отсутствие или наличие задолженностей по кредитам не обнаружено. Выдвинутые гипотезы не нашли значимого подтверждения. Возможно, необходима комплексная оценка влияния выделенных признаков.